In [27]:
import os
import yaml
import sys
sys.path.append('.')
from nbsignature import *
#from cwl import *
from clt import CommandLineTool
from wf import Workflow
from aninput import AnInput

In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
signature = get_signature_notebook('/workspace/burned-area/repo/burned-area/src/package/ades/test1.ipynb')

In [10]:
signature['_workflow']

{'label': 'Normalized burn ratio',
 'doc': 'Normalized burn ratio for burned area intensity assessment',
 'id': 'nbr'}

In [11]:
signature['_requirements']

{'ramMin': 16000}

In [12]:
signature['_parameters']

{'input_reference_scatter': {'doc': 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass',
  'id': 'input_reference_scatter',
  'max_occurs': '16',
  'label': 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)',
  'stac:collection': 'post_event',
  'stac:href': '/workspace/slstr/catalog.json',
  'value': 'https://catalog.terradue.com/sentinel3/search?format=json&amp;uid=S3B_SL_2_LST____20200613T070835_20200613T071135_20200613T085802_0179_040_063_2880_LN2_O_NR_004,https://catalog.terradue.com/sentinel3/search?format=json&amp;uid=S3B_SL_2_LST____20200613T070835_20200613T071135_20200613T085802_0179_040_063_2880_LN2_O_NR_004',
  'type': 'Directory',
  'scatter': 'True'},
 'tiling_level': {'id': 'tiling_level',
  'label': 'Tiling level',
  'doc': 'Tiling level',
  'type': 'string',
  'value': '5'},
 'false_color_1': {'id': 'false_color_1',
  'label': 'False Color 1 (S5, S3, S2)',
  'doc': 'False Color RGB composite using bands S5, S3 and S2

In [13]:
os.environ['PREFIX'] = 'aaa'
executable = 'my_exe'

In [38]:
clt_inputs = dict()
wf_inputs = dict()
step_inputs = dict()

for index, key in enumerate(list(signature['_parameters'].keys())):

    an_input = AnInput(signature, key, index)
    
    clt_inputs['inp{}'.format(index+1)] = an_input.get_clt_input()
    wf_inputs[key] = an_input.get_wf_input()
    step_inputs['inp{}'.format(index+1)] = key

In [39]:
wf = Workflow(signature)

wf.set_inputs(wf_inputs)
wf.set_steps(step_inputs)

In [40]:
clt = CommandLineTool(signature, executable, docker=None)

clt.set_inputs(clt_inputs)

In [41]:
cwl = dict()

cwl['cwlVersion'] = 'v1.0'

cwl['$graph'] = [clt.to_dict(), wf.to_dict()]

In [42]:
yaml.dump(cwl,
          sys.stdout,
          default_flow_style=False) 

$graph:
- baseCommand: my_exe
  class: CommandLineTool
  id: clt
  inputs:
    inp1:
      inputBinding:
        position: 1
        prefix: --input_reference_scatter
      type: Directory
    inp2:
      inputBinding:
        position: 2
        prefix: --tiling_level
      type: string
    inp3:
      inputBinding:
        position: 3
        prefix: --false_color_1
      type: enum
  outputs:
    results:
      outputBinding:
        glob: .
      type: Directory
  requirements:
    EnvVarRequirement:
      envDef:
        PATH: aaa/bin:/opt/anaconda/bin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/share/java/maven/bin:/opt/anaconda/bin:/opt/anaconda/condabin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/lib64/qt-3.3/bin:/usr/share/java/maven/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/fbrito/.local/bin:/home/fbrito/bin
        PREFIX: aaa
    ResourceRequirement:
      ramMin: 16000
  stderr: std.err
  stdout: std.out
- class: Workflow
  doc: Norm

In [37]:
os.getcwd()

'/workspace/repo2cli/src/module/cookiecutter-nb-blueprint/{{cookiecutter.project_slug}}/src/package/ades'

In [35]:
def default_params(signature, scatter_on=None):
    
    wf = Workflow(signature)

    is_scatter = wf.is_scatter()
    
    scatter_input = wf.get_scatter_input()
    
    defaults = dict()

    parameters = signature['_parameters']
    
    for key in parameters.keys():


        if 'min_occurs' in parameters[key].keys():

            defaults[key] = parameters[key]['value']

        if 'max_occurs' in parameters[key].keys():

            if parameters[key]['max_occurs'] == '1':

                defaults[key] = parameters[key]['value']

            else:

                defaults[key] = parameters[key]['value'].split(',')

        else:
            defaults[key] = parameters[key]['value']

    return defaults

In [31]:
default_params(signature)

{'input_reference_scatter': ['https://catalog.terradue.com/sentinel3/search?format=json&amp;uid=S3B_SL_2_LST____20200613T070835_20200613T071135_20200613T085802_0179_040_063_2880_LN2_O_NR_004',
  'https://catalog.terradue.com/sentinel3/search?format=json&amp;uid=S3B_SL_2_LST____20200613T070835_20200613T071135_20200613T085802_0179_040_063_2880_LN2_O_NR_004'],
 'tiling_level': '5',
 'false_color_1': 'Yes'}

In [32]:
wf1 = Workflow(signature)

In [33]:
wf1.is_scatter()

True

In [34]:
wf1.get_scatter_input()

'input_reference_scatter'

In [21]:
class CommandLineTool():
    
    def __init__(self, signature, executable, docker=None):
        
        self._clt_class = dict()
        self.signature = signature
        self.docker = docker
        
                
        self._clt_class['id'] = 'clt'
    
        if docker is not None:

            self._clt_class['hints'] = {'DockerRequirement': {'dockerPull': self.docker}}

        self._clt_class['class'] = 'CommandLineTool'
        self._clt_class['baseCommand'] = self.executable
        self._clt_class['stdout'] = 'std.out'
        self._clt_class['stderr'] = 'std.err'

        self._clt_class['outputs'] = {'results': {'outputBinding': { 'glob': '.'},
                                            'type': 'Directory'}} 

        self._clt_class['stdout'] = 'std.out'
        self._clt_class['stderr'] = 'std.err'

        env_vars = {'PATH' : self.get_path()}

        if 'PREFIX' in os.environ:

            env_vars['PREFIX'] = os.environ['PREFIX']



        self._clt_class['requirements'] = {'ResourceRequirement': self.signature['_requirements'] if '_requirements' in self.signature.keys() else {},
                                     'EnvVarRequirement' : {'envDef':
                                                            env_vars}}

    def set_inputs(self, inputs):
        
        self._clt_class['inputs'] = inputs

    def set_outputs(self, outputs):
        
        self._clt_class['outputs'] = outputs
        
    def to_dict(self):
        
        return self._clt_class

    def get_path(self):
    
        path = os.environ['PATH']

        if ';' in path:

            path = os.environ['PATH'].split(';')[1]

        if 'PREFIX' in os.environ:

            path = ':'.join([os.path.join(os.environ['PREFIX'], 'bin'), path])

        return path

In [24]:
clt_class = CommandLineTool(signature, executable, 'eoepca/burned-area:0.1')

In [25]:
clt_class.to_dict()

{'id': 'clt',
 'hints': {'DockerRequirement': {'dockerPull': 'eoepca/burned-area:0.1'}},
 'class': 'CommandLineTool',
 'baseCommand': 'my_exe',
 'stdout': 'std.out',
 'stderr': 'std.err',
 'outputs': {'results': {'outputBinding': {'glob': '.'}, 'type': 'Directory'}},
 'requirements': {'ResourceRequirement': {'ramMin': 16000},
  'EnvVarRequirement': {'envDef': {'PATH': 'aaa/bin:/opt/anaconda/bin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/share/java/maven/bin:/opt/anaconda/bin:/opt/anaconda/condabin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/lib64/qt-3.3/bin:/usr/share/java/maven/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/fbrito/.local/bin:/home/fbrito/bin',
    'PREFIX': 'aaa'}}}}

In [139]:
class Workflow():
    
    def __init__(self, signature):
        
        self._wf_class = dict()
        self.signature = signature

        self._wf_class['id'] = self.signature['_workflow']['id']
        self._wf_class['label'] = self.signature['_workflow']['label']
        self._wf_class['doc'] = self.signature['_workflow']['doc']
        self._wf_class['class'] = 'Workflow'
        
        self._wf_class['outputs'] = [{'id': 'wf_outputs',
                                      'outputSource': ['node_1/results'],
                                      'type': {'type': 'array',
                                         'items': 'Directory'}}]
        
        if self.is_scatter():
            
            self.scatter_param = self.get_scatter_input()
        
        else:
            
            self.scatter_param = None
            
        self._wf_class['inputs'] = None

    def get_scatter_input(self):
        
        scatter_input = None
        
        for index, key in enumerate(list(self.signature['_parameters'].keys())):
            
            if 'scatter' in self.signature['_parameters'][key].keys():
            
                scatter_input = key
        
                break
            
        return scatter_input
        
    def is_scatter(self):
        
        scatter = False
        
        for index, key in enumerate(list(self.signature['_parameters'].keys())):
            
            if 'scatter' in self.signature['_parameters'][key].keys():
            
                scatter = True
        
                break
            
        return scatter

    def set_inputs(self, inputs):
        
        self._wf_class['inputs'] = inputs

    def set_outputs(self, outputs):
        
        self._wf_class['outputs'] = outputs
        
    def set_steps(self, step_inputs):
        
        if self._wf_class['inputs'] is None:
            raise ValueError
        
        if self.is_scatter():
            
            self._wf_class['requirements'] = [{'class': 'ScatterFeatureRequirement'}]

            self._wf_class['steps'] = {'node_1': {'scatter': self.get_scatter_input(),
                                            'scatterMethod': 'dotproduct',
                                            'in': step_inputs,
                                            'out': ['results'],
                                            'run': '#clt'
                                      }
                            }

        
    def to_dict(self):
        
        return self._wf_class

In [140]:
class AnInput():

    def __init__(self, signature, key, index):
        
        self._input = dict()
        
        self._signature = signature
        self._key = key
        self._index = index
        
        self.input_signature = self._signature['_parameters'][self._key]
        
        self.input_type = self.get_param_type()
        
    def get_param_type(self):
    
        param_type = None

        if 'type' in self.input_signature:

            return self.input_signature['type']

        if not all(elem in list(self.input_signature) for elem in ['min_occurs', 'max_occurs']):

            # if not set, add the key and its default value

            if not 'min_occurs' in self.input_signature.keys():

                self.input_signature['min_occurs'] = 1

            if not 'max_occurs' in self.input_signature.keys():

                self.input_signature['max_occurs'] = 2

        # cast to int
        self.input_signature['min_occurs'] = int(self.input_signature['min_occurs']) 
        self.input_signature['max_occurs'] = int(self.input_signature['max_occurs'])

        if (self.input_signature['min_occurs'] == 0) & (self.input_signature['max_occurs'] == 1):

            param_type = 'string?'

        if (self.input_signature['min_occurs'] == 0) & (self.input_signature['max_occurs'] > 1):

            param_type = 'string[]?'

        if (self.input_signature['min_occurs'] == 1) & (self.input_signature['max_occurs'] == 1):

            param_type = 'string'

        if (self.input_signature['min_occurs'] == 1) & (self.input_signature['max_occurs'] > 1):

            param_type = 'string[]'

        if (self.input_signature['min_occurs'] > 1) & (self.input_signature['max_occurs'] > 1):

            param_type = 'string[]'

        return param_type 
    
    def parse_input(self):
        
        wf_input = None
        clt_input = None
        
        if self.input_type == 'enum' and 'symbols' in self.input_signature.keys():
                
            wf_input = {'type': {'type': self.input_type, 
                                         'symbols': self._signature['_parameters'][self._key]['symbols']},
                                'label': self._signature['_parameters'][self._key]['label'],
                                'doc': self._signature['_parameters'][self._key]['doc']}
            
        else: 
        
            if 'scatter' in self._signature['_parameters'][self._key].keys():

                if self._signature['_parameters'][self._key]['scatter'] == 'True':
                    scatter_on = self._key
                    scatter_input = 'inp{}'.format(self._index+1)

                    # the Workflow gets the type set in the notebook transformed as an array
                    wf_input = {'type': '{}[]'.format(self._signature['_parameters'][self._key]['type']),
                                        'label': self._signature['_parameters'][self._key]['label'],
                                        'doc': self._signature['_parameters'][self._key]['doc']
                               }
            else:

                wf_input = {'type': self.input_type,
                                    'label': self._signature['_parameters'][self._key]['label'],
                                    'doc': self._signature['_parameters'][self._key]['doc']
                           }
    
        clt_input = {'type': self.input_type,
                     'inputBinding': {'position': int(self._index+1),
                                      'prefix': '--{}'.format(self._key)}
                    }
    
     
        return wf_input, clt_input
    
    def get_wf_input(self):
        
        return self.parse_input()[0]
    
    def get_clt_input(self):
        
        return self.parse_input()[1]

In [141]:
clt_inputs = dict()
wf_inputs = dict()
step_inputs = dict()

for index, key in enumerate(list(signature['_parameters'].keys())):

    an_input = AnInput(signature, key, index)
    
    clt_inputs['inp{}'.format(index+1)] = an_input.get_clt_input()
    wf_inputs[key] = an_input.get_wf_input()
    step_inputs['inp{}'.format(index+1)] = key

In [142]:
clt_inputs

{'inp1': {'type': 'Directory',
  'inputBinding': {'position': 1, 'prefix': '--input_reference_scatter'}},
 'inp2': {'type': 'string',
  'inputBinding': {'position': 2, 'prefix': '--tiling_level'}},
 'inp3': {'type': 'enum',
  'inputBinding': {'position': 3, 'prefix': '--false_color_1'}}}

In [143]:
wf_inputs

{'input_reference_scatter': {'type': 'Directory[]',
  'label': 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)',
  'doc': 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass'},
 'tiling_level': {'type': 'string',
  'label': 'Tiling level',
  'doc': 'Tiling level'},
 'false_color_1': {'type': {'type': 'enum', 'symbols': ['Yes', 'No']},
  'label': 'False Color 1 (S5, S3, S2)',
  'doc': 'False Color RGB composite using bands S5, S3 and S2'}}

In [144]:
step_inputs

{'inp1': 'input_reference_scatter',
 'inp2': 'tiling_level',
 'inp3': 'false_color_1'}

In [145]:
wf = Workflow(signature)

wf.set_inputs(wf_inputs)
wf.set_steps(step_inputs)

In [146]:
wf.is_scatter()

True

In [147]:
wf.get_scatter_input()

'input_reference_scatter'

In [148]:
wf.to_dict()

{'id': 'nbr',
 'label': 'Normalized burn ratio',
 'doc': 'Normalized burn ratio for burned area intensity assessment',
 'class': 'Workflow',
 'outputs': [{'id': 'wf_outputs',
   'outputSource': ['node_1/results'],
   'type': {'type': 'array', 'items': 'Directory'}}],
 'inputs': {'input_reference_scatter': {'type': 'Directory[]',
   'label': 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)',
   'doc': 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass'},
  'tiling_level': {'type': 'string',
   'label': 'Tiling level',
   'doc': 'Tiling level'},
  'false_color_1': {'type': {'type': 'enum', 'symbols': ['Yes', 'No']},
   'label': 'False Color 1 (S5, S3, S2)',
   'doc': 'False Color RGB composite using bands S5, S3 and S2'}},
 'requirements': [{'class': 'ScatterFeatureRequirement'}],
 'steps': {'node_1': {'scatter': 'input_reference_scatter',
   'scatterMethod': 'dotproduct',
   'in': {'inp1': 'input_reference_scatter',
    'inp2': 'til

In [149]:
clt = CommandLineTool(signature, executable, docker=None)

In [150]:
clt.set_inputs(clt_inputs)

clt.to_dict()

{'id': 'clt',
 'class': 'CommandLineTool',
 'baseCommand': 'my_exe',
 'stdout': 'std.out',
 'stderr': 'std.err',
 'outputs': {'results': {'outputBinding': {'glob': '.'}, 'type': 'Directory'}},
 'requirements': {'ResourceRequirement': {'ramMin': 16000},
  'EnvVarRequirement': {'envDef': {'PATH': 'aaa/bin:/opt/anaconda/bin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/share/java/maven/bin:/opt/anaconda/bin:/opt/anaconda/condabin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/lib64/qt-3.3/bin:/usr/share/java/maven/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/fbrito/.local/bin:/home/fbrito/bin',
    'PREFIX': 'aaa'}}},
 'inputs': {'inp1': {'type': 'Directory',
   'inputBinding': {'position': 1, 'prefix': '--input_reference_scatter'}},
  'inp2': {'type': 'string',
   'inputBinding': {'position': 2, 'prefix': '--tiling_level'}},
  'inp3': {'type': 'enum',
   'inputBinding': {'position': 3, 'prefix': '--false_color_1'}}}}

In [151]:
cwl = dict()

cwl['cwlVersion'] = 'v1.0'

cwl['$graph'] = [clt.to_dict(), wf.to_dict()]


In [152]:
print(cwl)

{'cwlVersion': 'v1.0', '$graph': [{'id': 'clt', 'class': 'CommandLineTool', 'baseCommand': 'my_exe', 'stdout': 'std.out', 'stderr': 'std.err', 'outputs': {'results': {'outputBinding': {'glob': '.'}, 'type': 'Directory'}}, 'requirements': {'ResourceRequirement': {'ramMin': 16000}, 'EnvVarRequirement': {'envDef': {'PATH': 'aaa/bin:/opt/anaconda/bin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/share/java/maven/bin:/opt/anaconda/bin:/opt/anaconda/condabin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/lib64/qt-3.3/bin:/usr/share/java/maven/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/fbrito/.local/bin:/home/fbrito/bin', 'PREFIX': 'aaa'}}}, 'inputs': {'inp1': {'type': 'Directory', 'inputBinding': {'position': 1, 'prefix': '--input_reference_scatter'}}, 'inp2': {'type': 'string', 'inputBinding': {'position': 2, 'prefix': '--tiling_level'}}, 'inp3': {'type': 'enum', 'inputBinding': {'position': 3, 'prefix': '--false_color_1'}}}}, {'id': 'nbr', 'label': 'Normal

In [153]:
yaml.dump(cwl, 
                  sys.stdout, 
                  default_flow_style=False)


$graph:
- baseCommand: my_exe
  class: CommandLineTool
  id: clt
  inputs:
    inp1:
      inputBinding:
        position: 1
        prefix: --input_reference_scatter
      type: Directory
    inp2:
      inputBinding:
        position: 2
        prefix: --tiling_level
      type: string
    inp3:
      inputBinding:
        position: 3
        prefix: --false_color_1
      type: enum
  outputs:
    results:
      outputBinding:
        glob: .
      type: Directory
  requirements:
    EnvVarRequirement:
      envDef:
        PATH: aaa/bin:/opt/anaconda/bin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/share/java/maven/bin:/opt/anaconda/bin:/opt/anaconda/condabin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/lib64/qt-3.3/bin:/usr/share/java/maven/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/fbrito/.local/bin:/home/fbrito/bin
        PREFIX: aaa
    ResourceRequirement:
      ramMin: 16000
  stderr: std.err
  stdout: std.out
- class: Workflow
  doc: Norm

In [65]:
an_input.get_input()

({'type': {'type': 'enum', 'symbols': ['Yes', 'No']},
  'label': 'False Color 1 (S5, S3, S2)',
  'doc': 'False Color RGB composite using bands S5, S3 and S2'},
 {'type': 'enum',
  'inputBinding': {'position': 3, 'prefix': '--false_color_1'}})

In [47]:
clt_inputs = dict()
wf_inputs = dict()

for index, key in enumerate(list(signature['_parameters'].keys())):
    
    input_type = get_param_type(signature['_parameters'][key])
    
    print(index, key, input_type)
    
    # enum
    
    if input_type == 'enum' and 'symbols' in signature['_parameters'][key].keys():
                
            wf_inputs[key] = {'type': {'type': input_type, 
                                         'symbols': signature['_parameters'][key]['symbols']},
                                'label': signature['_parameters'][key]['label'],
                                'doc': signature['_parameters'][key]['doc']}
    
    else: 
        
        if 'scatter' in signature['_parameters'][key].keys():
                    
            if signature['_parameters'][key]['scatter'] == 'True':
                scatter_on = key
                scatter_input = 'inp{}'.format(index+1)

                # the Workflow gets the type set in the notebook transformed as an array
                wf_inputs[key] = {'type': '{}[]'.format(signature['_parameters'][key]['type']),
                                    'label': signature['_parameters'][key]['label'],
                                    'doc': signature['_parameters'][key]['doc']}
        else:

            wf_inputs[key] = {'type': input_type,
                                'label': signature['_parameters'][key]['label'],
                                'doc': signature['_parameters'][key]['doc']}
    
    clt_inputs['inp{}'.format(index+1)] = {'type': input_type,
                                                       'inputBinding': {'position': int(index+1),
                                                                        'prefix': '--{}'.format(key)}
                                                      }

0 input_reference_scatter Directory
1 tiling_level string
2 false_color_1 enum


In [48]:
wf_inputs

{'input_reference_scatter': {'type': 'Directory[]',
  'label': 'Sentinel-3 SLSTR Level-2 (SL_2_LST____ descending pass)',
  'doc': 'This service takes as input a Sentinel-3 SLSTR Level 2 (SL_2_LST____) product on DESCENDING pass'},
 'tiling_level': {'type': 'string',
  'label': 'Tiling level',
  'doc': 'Tiling level'},
 'false_color_1': {'type': {'type': 'enum', 'symbols': ['Yes', 'No']},
  'label': 'False Color 1 (S5, S3, S2)',
  'doc': 'False Color RGB composite using bands S5, S3 and S2'}}

In [49]:
clt_inputs

{'inp1': {'type': 'Directory',
  'inputBinding': {'position': 1, 'prefix': '--input_reference_scatter'}},
 'inp2': {'type': 'string',
  'inputBinding': {'position': 2, 'prefix': '--tiling_level'}},
 'inp3': {'type': 'enum',
  'inputBinding': {'position': 3, 'prefix': '--false_color_1'}}}

In [12]:
yaml.dump(cwl(signature, 'my_exe'),
          sys.stdout,
          default_flow_style=False) 

$graph:
- baseCommand: my_exe
  class: CommandLineTool
  id: clt
  inputs:
    inp1:
      inputBinding:
        position: 1
        prefix: --input_reference_scatter
      type: Directory
    inp2:
      inputBinding:
        position: 2
        prefix: --tiling_level
      type: string
    inp3:
      inputBinding:
        position: 3
        prefix: --false_color_1
      type: enum
  outputs:
    results:
      outputBinding:
        glob: .
      type: Any
  requirements:
    EnvVarRequirement:
      envDef:
        PATH: aaa/bin:/opt/anaconda/envs/env_s3/conda-otb/bin:/opt/anaconda/envs/env_s3/snap/bin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/envs/env_s3/bin:/usr/share/java/maven/bin:/opt/anaconda/bin:/opt/anaconda/condabin:/opt/anaconda/envs/notebook/bin:/opt/anaconda/bin:/usr/lib64/qt-3.3/bin:/usr/share/java/maven/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/fbrito/.local/bin:/home/fbrito/bin
        PREFIX: aaa
    ResourceRequirement:
      ramMin: 16000
  

In [13]:
cwl_file = 'workflow.cwl'

In [14]:
with open(cwl_file, 'w') as file:
    yaml.dump(cwl(signature, 'my_exe'), file)